# Online Gradient Boosting Decision Tree (OnlineGBDT) Demo
This notebook shows how to train, update, and evaluate the custom `DecisionTree` and `OnlineGBDT` classes.

👉 *Make sure the module that defines `DecisionTree` and `OnlineGBDT` is importable (e.g. saved as **`online_gbdt.py`** in the same folder or installed as a package).*

In [3]:
import os
import sys

# Get the current working directory
current_directory = os.getcwd()

# List all files and directories in the current directory
files_and_directories = os.listdir(current_directory)

# Print the list of files and directories
print("Files and directories in the current directory:")
for item in files_and_directories:
    print(item)


Files and directories in the current directory:
online_gbdt_demo.ipynb


In [4]:
import numpy as np
import matplotlib.pyplot as plt

from fogo.online_gbdt import DecisionTree, OnlineGBDT  # or paste the class definitions here

ModuleNotFoundError: No module named 'fogo'

In [ ]:
# Create a simple synthetic regression dataset
np.random.seed(42)
N = 200
X = np.random.randn(N, 5)
true_coef = np.array([3, -2, 0, 1, 2])
y = X @ true_coef + np.random.randn(N) * 0.5
print('Dataset shape:', X.shape)

In [ ]:
# Train the gradient‑boosting ensemble
model = OnlineGBDT(n_estimators=10, learning_rate=0.1, max_depth=3)
model.fit(X, y)
print('Initial training complete.')

In [ ]:
# Evaluate training MSE
pred = model.predict(X)
mse = np.mean((y - pred) ** 2)
print(f'Mean squared error: {mse:.4f}')

In [ ]:
# --- Incremental learning example (fit_one) ---
x_new = np.random.randn(5)
y_new = x_new @ true_coef + np.random.randn() * 0.5
print('Prediction before update:', model.predict([x_new])[0])
model.fit_one(x_new, y_new)
print('Prediction after update :', model.predict([x_new])[0])

In [ ]:
# --- Decremental learning example ---
residual = y_new - model.predict([x_new])[0]
model.decrement(x_new, residual)
print('Prediction after decrement:', model.predict([x_new])[0])

In [ ]:
# Visualise predictions vs. ground truth
plt.figure(figsize=(5,5))
plt.scatter(y, pred, alpha=0.6)
plt.plot([y.min(), y.max()], [y.min(), y.max()])
plt.xlabel('True')
plt.ylabel('Predicted')
plt.title('OnlineGBDT predictions')
plt.show()